# Import Library

In [21]:
import pandas as pd
import re
import nltk
import torch

from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Read Dataset

- Impor seluruh dataset

In [22]:
df = pd.read_csv('./data/hukum_online_pidana.csv')
df.head()

,question,answer
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...


In [23]:
df2 = pd.read_csv('./data/hukum_online_tech.csv')
df2.head()

,question,answer
0,"Dengan maraknya AI, mahasiswa sekarang lebih m...",Terima kasih atas pertanyaan Anda.Apa itu Arti...
1,Belum lama ini salah satu aplikasi e-commerce ...,Terima kasih atas pertanyaan Anda.Salah satu a...
2,"Saya izin bertanya, saya bersekolah dimana SMA...",Terima kasih atas pertanyaan Anda.Seluruh info...
3,Saat ini telah berkembang berbagai tren aplika...,Terima kasih atas pertanyaan Anda.Artikel di b...
4,Saya ingin bertanya tentang orang yang ikut me...,Terima kasih atas pertanyaan Anda. KLINIK TERK...


- Gabungkan dataset

In [24]:
new_df= pd.concat([df, df2], ignore_index=True)
new_df.head()

,question,answer
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...


- Melihat informasi Dataset

In [25]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747 entries, 0 to 1746
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1747 non-null   object
 1   answer    1747 non-null   object
dtypes: object(2)
memory usage: 27.4+ KB


# Data Cleaning

### Ekstrak Teks

In [26]:
def ekstrak_pasal(teks):
    """
    Mengekstrak semua teks pasal dalam teks yang diberikan.

    Args:
      teks: Teks yang ingin dicari.

    Returns:
      List dari teks pasal yang ditemukan.
    """
    pola_pasal = r"Pasal\s+\w+(?:\s+ayat\s*\(\d+\w*\))?\s+(?:KUHP|UU\s+ITE)|Pasal\s+\d+\s+UU\s+\d+/\d{4}"
    pasal = re.findall(pola_pasal, teks)
    return  pasal

In [27]:
new_df['pasal'] = new_df['answer'].apply(ekstrak_pasal)
new_df

,question,answer,pasal
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 290 KUHP, Pasal 415 UU 1/2023, Pasal 41..."
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 56 KUHP, Pasal 160 KUHP, Pasal 160 KUHP..."
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 263 KUHP, Pasal 391 UU 1/2023, Pasal 26..."
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...,"[Pasal 3 UU 31/1999, Pasal 372 KUHP, Pasal 486..."
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...,"[Pasal 48 KUHP, Pasal 48 KUHP, Pasal 42 UU 1/2..."
...,...,...,...
1742,Cybersquater itu seperti apa dan bagaimana? Ap...,Cybersquatter asal katanya dari cyberquatting....,[]
1743,"Dalam menghadapi globalisasi TI, faktor-faktor...","Sdri Mega yang terhormat. Sebelum lebih jauh, ...",[]
1744,1.Bagaimanakah pengaturan mengenai cyber crime...,Dari pertanyaan yang anda ajukan kami berasums...,[]
1745,Apakah Indonesia memiliki Undang-undang tentan...,Pertanyaan ini sering diajukan juga oleh beber...,[]


### Hapus Data Pasal yang Kosong

In [28]:
jumlah_list_kosong = len(new_df[new_df['pasal'].apply(lambda x: len(x) == 0)])
print(jumlah_list_kosong)

769


In [29]:
new_df = new_df[new_df['pasal'].apply(lambda x: len(x) > 0)] 
new_df

,question,answer,pasal
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 290 KUHP, Pasal 415 UU 1/2023, Pasal 41..."
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 56 KUHP, Pasal 160 KUHP, Pasal 160 KUHP..."
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 263 KUHP, Pasal 391 UU 1/2023, Pasal 26..."
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...,"[Pasal 3 UU 31/1999, Pasal 372 KUHP, Pasal 486..."
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...,"[Pasal 48 KUHP, Pasal 48 KUHP, Pasal 42 UU 1/2..."
...,...,...,...
1726,"Permisi pak. Menurut yang saya baca, Apple di ...","Secara sederhana, jailbreakingialah modifikas...","[Pasal 25 UU ITE, Pasal 25 UU ITE]"
1728,Bagaimana sanksi jika negara lain menyadap kom...,"Sahabat hukumonline yang baik, KLINIK TERKAITM...","[Pasal 31 UU ITE, Pasal 47 UU ITE, Pasal 31 ay..."
1730,Apakah salah jika kita upload foto seseorang a...,Terima kasih atas pertanyaan Anda. KLINIK TERK...,"[Pasal 27 UU ITE, Pasal 27 UU ITE]"
1734,Apakah BBM (Blackberry Messenger) termasuk ke ...,"Yth. Pembaca hukumonline,KLINIK TERKAITJerat P...",[Pasal 27 ayat (3) UU ITE]


### Bersihkan Data pada Kolom Pasal

In [30]:
new_df['pasal'] = new_df['pasal'].apply(lambda pasal_list: [pasal.replace('\xa0', ' ') for pasal in pasal_list])
new_df

C:\Users\Wahpram\AppData\Local\Temp\ipykernel_12236\1437179267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['pasal'] = new_df['pasal'].apply(lambda pasal_list: [pasal.replace('\xa0', ' ') for pasal in pasal_list])


,question,answer,pasal
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 290 KUHP, Pasal 415 UU 1/2023, Pasal 41..."
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 56 KUHP, Pasal 160 KUHP, Pasal 160 KUHP..."
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 263 KUHP, Pasal 391 UU 1/2023, Pasal 26..."
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...,"[Pasal 3 UU 31/1999, Pasal 372 KUHP, Pasal 486..."
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...,"[Pasal 48 KUHP, Pasal 48 KUHP, Pasal 42 UU 1/2..."
...,...,...,...
1726,"Permisi pak. Menurut yang saya baca, Apple di ...","Secara sederhana, jailbreakingialah modifikas...","[Pasal 25 UU ITE, Pasal 25 UU ITE]"
1728,Bagaimana sanksi jika negara lain menyadap kom...,"Sahabat hukumonline yang baik, KLINIK TERKAITM...","[Pasal 31 UU ITE, Pasal 47 UU ITE, Pasal 31 ay..."
1730,Apakah salah jika kita upload foto seseorang a...,Terima kasih atas pertanyaan Anda. KLINIK TERK...,"[Pasal 27 UU ITE, Pasal 27 UU ITE]"
1734,Apakah BBM (Blackberry Messenger) termasuk ke ...,"Yth. Pembaca hukumonline,KLINIK TERKAITJerat P...",[Pasal 27 ayat (3) UU ITE]


### Mengambil Data Pasal yang lebih dari 20

- Melihat Jumlah Pasal yang Hanya Memiliki Satu Data

In [31]:
all_pasal = [pasal for sublist in new_df['pasal'] for pasal in sublist]

unique_pasal = set(all_pasal)

# Menampilkan jumlah dan pasal unik
print(f"Jumlah pasal unik: {len(unique_pasal)}")
print(f"Pasal unik: {unique_pasal}")

Jumlah pasal unik: 844
Pasal unik: {'Pasal 18 UU 2/2002', 'Pasal 78 UU 3/2011', 'Pasal 8 UU 20/2021', 'Pasal 439 UU 1/2023', 'Pasal 3 UU 31/1999', 'Pasal 68 UU 24/2009', 'Pasal 30 KUHP', 'Pasal 77 UU 35/2014', 'Pasal 42 UU ITE', 'Pasal 411 UU 1/2023', 'Pasal 362 KUHP', 'Pasal 3 UU 1/2023', 'Pasal 263 ayat (1) KUHP', 'Pasal 344 KUHP', 'Pasal 30 UU 1/2023', 'Pasal 83 KUHP', 'Pasal 4 UU 14/2015', 'Pasal 264 UU 1/2023', 'Pasal 480 ayat (2) KUHP', 'Pasal 68 UU 35/2014', 'Pasal 204 ayat (1) KUHP', 'Pasal 27 UU 8/2009', 'Pasal 3 UU 21/2007', 'Pasal 195 UU 42/2008', 'Pasal 55 KUHP', 'Pasal 63 KUHP', 'Pasal 1 ayat (1) UU ITE', 'Pasal 11 UU 1/2006', 'Pasal 80 UU 17/2014', 'Pasal 317 KUHP', 'Pasal 437 UU 1/2009', 'Pasal 27 ayat (1)  UU ITE', 'Pasal 84 KUHP', 'Pasal 521 UU 1/2023', 'Pasal 19 UU 15/2003', 'Pasal 21 UU 1/2023', 'Pasal 79 UU 11/2012', 'Pasal 502 UU 1/2023', 'Pasal 407 ayat (1) KUHP', 'Pasal 382 KUHP', 'Pasal 360 ayat (1) KUHP', 'Pasal 47 UU ITE', 'Pasal 355 KUHP', 'Pasal 351 ayat (3)

- Melihat Jumlah Pasal yang Lebih dari Satu Data

In [32]:
pasal_counts = Counter(all_pasal)

non_unique_pasal = {pasal: count for pasal, count in pasal_counts.items() if count > 1}

# Menampilkan pasal yang tidak unik
print(f"Pasal yang tidak unik: {non_unique_pasal}")
print(f"Jumlah: {len(non_unique_pasal)}")

Pasal yang tidak unik: {'Pasal 290 KUHP': 4, 'Pasal 415 UU 1/2023': 11, 'Pasal 292 KUHP': 17, 'Pasal 417 UU 1/2023': 7, 'Pasal 414 UU 1/2023': 14, 'Pasal 65 ayat (1) KUHP': 7, 'Pasal 56 KUHP': 19, 'Pasal 160 KUHP': 13, 'Pasal 246 UU 1/2023': 3, 'Pasal 263 KUHP': 39, 'Pasal 391 UU 1/2023': 12, 'Pasal 263 ayat (2) KUHP': 6, 'Pasal 3 UU 31/1999': 16, 'Pasal 372 KUHP': 81, 'Pasal 486 UU 1/2023': 38, 'Pasal 374 KUHP': 21, 'Pasal 488 UU 1/2023': 10, 'Pasal 378 KUHP': 153, 'Pasal 492 UU 1/2023': 69, 'Pasal 48 KUHP': 16, 'Pasal 42 UU 1/2023': 3, 'Pasal 338 KUHP': 52, 'Pasal 351 ayat (3) KUHP': 8, 'Pasal 351 KUHP': 69, 'Pasal 78 KUHP': 27, 'Pasal 284 KUHP': 77, 'Pasal 284 ayat (1) KUHP': 11, 'Pasal 10 KUHP': 8, 'Pasal 65 UU 1/2023': 2, 'Pasal 351 ayat (1) KUHP': 39, 'Pasal 352 ayat (1) KUHP': 17, 'Pasal 79 UU 1/2023': 11, 'Pasal 624 UU 1/2023': 2, 'Pasal 1365 KUHP': 17, 'Pasal 285 KUHP': 38, 'Pasal 20 UU 14/2005': 2, 'Pasal 362 KUHP': 128, 'Pasal 476 UU 1/2023': 30, 'Pasal 271 UU 1/2023': 3, 'P

- Melihat Jumlah Data pada Pasal

In [33]:
for pasal, count in non_unique_pasal.items():
    print(f"Pasal: {pasal} | Jumlah kemunculan: {count}")
    
total_data = sum(non_unique_pasal.values())
print(f"\nJumlah total data dengan pasal tidak unik: {total_data}")

Pasal: Pasal 290 KUHP | Jumlah kemunculan: 4
Pasal: Pasal 415 UU 1/2023 | Jumlah kemunculan: 11
Pasal: Pasal 292 KUHP | Jumlah kemunculan: 17
Pasal: Pasal 417 UU 1/2023 | Jumlah kemunculan: 7
Pasal: Pasal 414 UU 1/2023 | Jumlah kemunculan: 14
Pasal: Pasal 65 ayat (1) KUHP | Jumlah kemunculan: 7
Pasal: Pasal 56 KUHP | Jumlah kemunculan: 19
Pasal: Pasal 160 KUHP | Jumlah kemunculan: 13
Pasal: Pasal 246 UU 1/2023 | Jumlah kemunculan: 3
Pasal: Pasal 263 KUHP | Jumlah kemunculan: 39
Pasal: Pasal 391 UU 1/2023 | Jumlah kemunculan: 12
Pasal: Pasal 263 ayat (2) KUHP | Jumlah kemunculan: 6
Pasal: Pasal 3 UU 31/1999 | Jumlah kemunculan: 16
Pasal: Pasal 372 KUHP | Jumlah kemunculan: 81
Pasal: Pasal 486 UU 1/2023 | Jumlah kemunculan: 38
Pasal: Pasal 374 KUHP | Jumlah kemunculan: 21
Pasal: Pasal 488 UU 1/2023 | Jumlah kemunculan: 10
Pasal: Pasal 378 KUHP | Jumlah kemunculan: 153
Pasal: Pasal 492 UU 1/2023 | Jumlah kemunculan: 69
Pasal: Pasal 48 KUHP | Jumlah kemunculan: 16
Pasal: Pasal 42 UU 1/2023

- Memfilter Pasal yang Memiliki Lebih dari 20 Data

In [36]:
filtered_pasal = {pasal: count for pasal, count in pasal_counts.items() if count > 20}

# Menampilkan hasil
print(f"Pasal yang memiliki jumlah kemunculan lebih dari 20: {filtered_pasal}")
print(f"Jumlah: {len(filtered_pasal)}")

# Menampilkan detail pasal dan jumlah kemunculannya
for pasal, count in filtered_pasal.items():
    print(f"Pasal: {pasal} | Jumlah kemunculan: {count}")

Pasal yang memiliki jumlah kemunculan lebih dari 20: {'Pasal 263 KUHP': 39, 'Pasal 372 KUHP': 81, 'Pasal 486 UU 1/2023': 38, 'Pasal 374 KUHP': 21, 'Pasal 378 KUHP': 153, 'Pasal 492 UU 1/2023': 69, 'Pasal 338 KUHP': 52, 'Pasal 351 KUHP': 69, 'Pasal 78 KUHP': 27, 'Pasal 284 KUHP': 77, 'Pasal 351 ayat (1) KUHP': 39, 'Pasal 285 KUHP': 38, 'Pasal 362 KUHP': 128, 'Pasal 476 UU 1/2023': 30, 'Pasal 311 ayat (1) KUHP': 35, 'Pasal 311 KUHP': 48, 'Pasal 310 ayat (1) KUHP': 89, 'Pasal 310 KUHP': 132, 'Pasal 433 UU 1/2023': 52, 'Pasal 315 KUHP': 109, 'Pasal 436 UU 1/2023': 51, 'Pasal 352 KUHP': 48, 'Pasal 466 UU 1/2023': 26, 'Pasal 480 KUHP': 55, 'Pasal 591 UU 1/2023': 23, 'Pasal 411 UU 1/2023': 24, 'Pasal 29 UU ITE': 41, 'Pasal 335 KUHP': 32, 'Pasal 406 KUHP': 28, 'Pasal 406 ayat (1) KUHP': 27, 'Pasal 406 ayat (2) KUHP': 25, 'Pasal 521 UU 1/2023': 21, 'Pasal 27 ayat (1) UU ITE': 69, 'Pasal 44 KUHP': 30, 'Pasal 289 KUHP': 34, 'Pasal 303 KUHP': 22, 'Pasal 170 KUHP': 25, 'Pasal 55 KUHP': 41, 'Pasal 3

In [37]:
valid_pasal = set(filtered_pasal.keys())

new_df.loc[:, 'pasal'] = new_df['pasal'].apply(lambda pasal_list: [pasal for pasal in pasal_list if pasal in valid_pasal])
new_df


,question,answer,pasal
0,Belakangan ini cukup banyak kasus sodomi bocah...,Terima kasih atas pertanyaan Anda.Artikel di b...,[]
1,Bisakah seseorang yang mengompori dan membiark...,Terima kasih atas pertanyaan Anda.Artikel di b...,[]
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
4,Bagaimana ketentuan overmacht dalam hukum pida...,KLINIK TERKAITDaya Paksa dan Pembelaan Terpaks...,[]
...,...,...,...
1726,"Permisi pak. Menurut yang saya baca, Apple di ...","Secara sederhana, jailbreakingialah modifikas...",[]
1728,Bagaimana sanksi jika negara lain menyadap kom...,"Sahabat hukumonline yang baik, KLINIK TERKAITM...",[]
1730,Apakah salah jika kita upload foto seseorang a...,Terima kasih atas pertanyaan Anda. KLINIK TERK...,[]
1734,Apakah BBM (Blackberry Messenger) termasuk ke ...,"Yth. Pembaca hukumonline,KLINIK TERKAITJerat P...",[Pasal 27 ayat (3) UU ITE]


In [38]:
all_pasal = [pasal for sublist in new_df['pasal'] for pasal in sublist]

unique_pasal = set(all_pasal)

pasal_counts = Counter(all_pasal)
non_unique_pasal = {pasal: count for pasal, count in pasal_counts.items() if count > 1}

In [39]:
for pasal, count in non_unique_pasal.items():
    print(f"Pasal: {pasal} | Jumlah kemunculan: {count}")
    
total_data = sum(non_unique_pasal.values())
print(f"\nJumlah total data dengan pasal tidak unik: {total_data}")

Pasal: Pasal 263 KUHP | Jumlah kemunculan: 39
Pasal: Pasal 372 KUHP | Jumlah kemunculan: 81
Pasal: Pasal 486 UU 1/2023 | Jumlah kemunculan: 38
Pasal: Pasal 374 KUHP | Jumlah kemunculan: 21
Pasal: Pasal 378 KUHP | Jumlah kemunculan: 153
Pasal: Pasal 492 UU 1/2023 | Jumlah kemunculan: 69
Pasal: Pasal 338 KUHP | Jumlah kemunculan: 52
Pasal: Pasal 351 KUHP | Jumlah kemunculan: 69
Pasal: Pasal 78 KUHP | Jumlah kemunculan: 27
Pasal: Pasal 284 KUHP | Jumlah kemunculan: 77
Pasal: Pasal 351 ayat (1) KUHP | Jumlah kemunculan: 39
Pasal: Pasal 285 KUHP | Jumlah kemunculan: 38
Pasal: Pasal 362 KUHP | Jumlah kemunculan: 128
Pasal: Pasal 476 UU 1/2023 | Jumlah kemunculan: 30
Pasal: Pasal 311 ayat (1) KUHP | Jumlah kemunculan: 35
Pasal: Pasal 311 KUHP | Jumlah kemunculan: 48
Pasal: Pasal 310 ayat (1) KUHP | Jumlah kemunculan: 89
Pasal: Pasal 310 KUHP | Jumlah kemunculan: 132
Pasal: Pasal 433 UU 1/2023 | Jumlah kemunculan: 52
Pasal: Pasal 315 KUHP | Jumlah kemunculan: 109
Pasal: Pasal 436 UU 1/2023 | J

- Menghapus Data Pasal yang Kosong

In [40]:
new_df = new_df[new_df['pasal'].apply(lambda x: len(x) > 0)] 
new_df

,question,answer,pasal
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,Apa pasal yang bisa menjerat pengurus yayasan ...,Terima kasih atas pertanyaan Anda.Bisakah Peny...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,"Belakangan ini, viral kasus seorang suami di S...",Terima kasih atas pertanyaan Anda.Artikel di b...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,Saya ingin menanyakan bila laki-laki dan perem...,Terima kasih atas pertanyaan Anda.Artikel di b...,[Pasal 78 KUHP]
7,Saya seorang pria yang sudah beristri. Sayangn...,Terima kasih atas pertanyaan Anda.Artikel ini ...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...,...
1704,Apakah teror melalui private number ke nomor H...,Artikel di bawah ini adalah pemutakhiran dari ...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
1711,Belakangan ini banyak beredar game yang berko...,KLINIK TERKAITHukumnya Memiliki dan Membawa A...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
1715,Apakah bisa seorang yang melakukan tindakan c...,KLINIK TERKAITHukumnya Mengunggah Foto Kue Ber...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
1724,Saya mendapat tawaran untuk membuat tampilan w...,"Rekan Hukum Online, Secara umum, Pasal 25 Unda...",[Pasal 35 UU ITE]


### Menghapus Kolom yang Tidak Digunakan

In [59]:
new_df = new_df.drop(columns='answer')

In [60]:
new_df

,question,pasal
2,Kawan saya bekerja pada sebuah perusahaan. Dul...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,Apa pasal yang bisa menjerat pengurus yayasan ...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,"Belakangan ini, viral kasus seorang suami di S...","[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,Saya ingin menanyakan bila laki-laki dan perem...,[Pasal 78 KUHP]
7,Saya seorang pria yang sudah beristri. Sayangn...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,Apakah teror melalui private number ke nomor H...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,Belakangan ini banyak beredar game yang berko...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,Apakah bisa seorang yang melakukan tindakan c...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,Saya mendapat tawaran untuk membuat tampilan w...,[Pasal 35 UU ITE]


# Data Preprocessing

### Casefolding

In [61]:
new_df['question'] = new_df['question'].astype(str).str.lower()
new_df

,question,pasal
2,kawan saya bekerja pada sebuah perusahaan. dul...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,apa pasal yang bisa menjerat pengurus yayasan ...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,"belakangan ini, viral kasus seorang suami di s...","[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,saya ingin menanyakan bila laki-laki dan perem...,[Pasal 78 KUHP]
7,saya seorang pria yang sudah beristri. sayangn...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,apakah teror melalui private number ke nomor h...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,belakangan ini banyak beredar game yang berko...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,apakah bisa seorang yang melakukan tindakan c...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,saya mendapat tawaran untuk membuat tampilan w...,[Pasal 35 UU ITE]


### Remove Punctuation

In [62]:
new_df['question'] = new_df['question'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
new_df

,question,pasal
2,kawan saya bekerja pada sebuah perusahaan dulu...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,apa pasal yang bisa menjerat pengurus yayasan ...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,belakangan ini viral kasus seorang suami di su...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,saya ingin menanyakan bila lakilaki dan peremp...,[Pasal 78 KUHP]
7,saya seorang pria yang sudah beristri sayangny...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,apakah teror melalui private number ke nomor h...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,belakangan ini banyak beredar game yang berko...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,apakah bisa seorang yang melakukan tindakan c...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,saya mendapat tawaran untuk membuat tampilan w...,[Pasal 35 UU ITE]


### Remove Stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('indonesian'))

exclude_words = ["dan", "serta"]

def remove_stopwords_with_exceptions(text):
    # Tokenisasi teks
    words = text.split()
    
    # Filter kata-kata, menghapus stopwords kecuali pengecualian
    filtered_words = [word for word in words if word not in stop_words or word in exclude_words]
    
    # Menggabungkan kembali kata-kata yang tersisa menjadi satu string
    return ' '.join(filtered_words)

new_df['question'] = new_df['question'].apply(remove_stopwords_with_exceptions)
new_df


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wahpram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wahpram\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,question,pasal
2,kawan perusahaan melamar usianya 30an salah sy...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,pasal menjerat pengurus yayasan filantropi men...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,viral suami sumenep celurit istri tewas berita...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,lakilaki dan perempuan berpacaran alasan suka ...,[Pasal 78 KUHP]
7,pria beristri sayangnya pernikahan tercatat ku...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,teror private number nomor hp tindak kriminal ...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,beredar game berkonten negatif pornografi meng...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,tindakan curang bermain game online cheatschea...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,tawaran tampilan website semirip websitea dibi...,[Pasal 35 UU ITE]


In [64]:
df_stem = new_df.copy()
df_stem

,question,pasal
2,kawan perusahaan melamar usianya 30an salah sy...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,pasal menjerat pengurus yayasan filantropi men...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,viral suami sumenep celurit istri tewas berita...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,lakilaki dan perempuan berpacaran alasan suka ...,[Pasal 78 KUHP]
7,pria beristri sayangnya pernikahan tercatat ku...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,teror private number nomor hp tindak kriminal ...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,beredar game berkonten negatif pornografi meng...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,tindakan curang bermain game online cheatschea...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,tawaran tampilan website semirip websitea dibi...,[Pasal 35 UU ITE]


### Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()


df_stem['question'] = df_stem['question'].apply(lambda x: stemmer.stem(x))
df_stem

,question,pasal
2,kawan usaha lamar usia 30an salah syarat usia ...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]"
3,pasal jerat urus yayasan filantropi menyalahgu...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37..."
5,viral suami sumenep celurit istri tewas berita...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH..."
6,lakilaki dan perempuan pacar alas suka suka hu...,[Pasal 78 KUHP]
7,pria istri sayang nikah catat kua nikah siri p...,"[Pasal 284 KUHP, Pasal 284 KUHP]"
...,...,...
962,teror private number nomor hp tindak kriminal ...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU..."
966,edar game konten negatif pornografi kandung un...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U..."
968,tindak curang main game online cheatscheater k...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH..."
971,tawar tampil website mirip websitea bilang nam...,[Pasal 35 UU ITE]


In [106]:
df_prepro = df_stem.copy()

### Mengubah List Pasal ke Dalam Bentuk Kolom

In [107]:
all_pasal = set(pasal for sublist in df_prepro['pasal'] for pasal in sublist)

# 2. Buat kolom untuk setiap pasal
for pasal in all_pasal:
    df_prepro[pasal] = df_prepro['pasal'].apply(lambda x: 1 if pasal in x else 0)

df_prepro

,question,pasal,Pasal 78 KUHP,Pasal 284 KUHP,Pasal 436 UU 1/2023,Pasal 591 UU 1/2023,Pasal 521 UU 1/2023,Pasal 311 KUHP,Pasal 351 ayat (1) KUHP,Pasal 170 KUHP,...,Pasal 330 KUHP,Pasal 363 KUHP,Pasal 362 KUHP,Pasal 310 KUHP,Pasal 406 ayat (1) KUHP,Pasal 335 KUHP,Pasal 492 UU 1/2023,Pasal 35 UU ITE,Pasal 378 KUHP,Pasal 310 ayat (1) KUHP
2,kawan usaha lamar usia 30an salah syarat usia ...,"[Pasal 263 KUHP, Pasal 263 KUHP, Pasal 263 KUHP]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pasal jerat urus yayasan filantropi menyalahgu...,"[Pasal 372 KUHP, Pasal 486 UU 1/2023, Pasal 37...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
5,viral suami sumenep celurit istri tewas berita...,"[Pasal 338 KUHP, Pasal 338 KUHP, Pasal 351 KUH...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,lakilaki dan perempuan pacar alas suka suka hu...,[Pasal 78 KUHP],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,pria istri sayang nikah catat kua nikah siri p...,"[Pasal 284 KUHP, Pasal 284 KUHP]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,teror private number nomor hp tindak kriminal ...,"[Pasal 29 UU ITE, Pasal 29 UU ITE, Pasal 29 UU...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966,edar game konten negatif pornografi kandung un...,"[Pasal 27 ayat (1) UU ITE, Pasal 45 ayat (1) U...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
968,tindak curang main game online cheatscheater k...,"[Pasal 378 KUHP, Pasal 362 KUHP, Pasal 378 KUH...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
971,tawar tampil website mirip websitea bilang nam...,[Pasal 35 UU ITE],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Menghapus Kolom yang Tidak Digunakan

In [108]:
df_prepro = df_prepro.drop(columns='pasal')
df_prepro

,question,Pasal 78 KUHP,Pasal 284 KUHP,Pasal 436 UU 1/2023,Pasal 591 UU 1/2023,Pasal 521 UU 1/2023,Pasal 311 KUHP,Pasal 351 ayat (1) KUHP,Pasal 170 KUHP,Pasal 27 ayat (3) UU ITE,...,Pasal 330 KUHP,Pasal 363 KUHP,Pasal 362 KUHP,Pasal 310 KUHP,Pasal 406 ayat (1) KUHP,Pasal 335 KUHP,Pasal 492 UU 1/2023,Pasal 35 UU ITE,Pasal 378 KUHP,Pasal 310 ayat (1) KUHP
2,kawan usaha lamar usia 30an salah syarat usia ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pasal jerat urus yayasan filantropi menyalahgu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
5,viral suami sumenep celurit istri tewas berita...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,lakilaki dan perempuan pacar alas suka suka hu...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,pria istri sayang nikah catat kua nikah siri p...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,teror private number nomor hp tindak kriminal ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966,edar game konten negatif pornografi kandung un...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
968,tindak curang main game online cheatscheater k...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
971,tawar tampil website mirip websitea bilang nam...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Membagi Data Training dan Validation

In [ ]:
# Memisahkan teks dan label
X = df_prepro['question']  # Kolom teks
y = df_prepro.drop('question', axis=1)  # Kolom pasal (label)

# Membagi data menjadi data latih dan validasi
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Melakukan Tokenisasi

In [ ]:
# Memuat tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('cahya/bert-base-indonesian-522M')

# Tokenisasi data latih dan validasi
train_encodings = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')


### Membuat Custom Dataset untuk Training dan Validation

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Mengambil data dan label pada index tertentu
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float32)  # Mengonversi label menjadi tensor
        return item

    def __len__(self):
        return len(self.labels)

# Membuat dataset untuk training dan validation
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_val)


### Membuat DataLoader untuk Training dan Validation

In [ ]:
# Membuat DataLoader untuk training dan validation
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

### Modeling

In [ ]:
# Memuat model BERT dengan jumlah label sesuai jumlah pasal yang ada
model = BertForSequenceClassification.from_pretrained('cahya/bert-base-indonesian-522M', 
                                                      num_labels=y_train.shape[1],  # Jumlah pasal
                                                      problem_type="multi_label_classification")

# Menyiapkan optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Wahpram\miniconda3\envs\env-TA\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [121]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Melakukan Proses Training Data

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        # Ambil batch dari data loader
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass dan optimasi
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss}")

    # Evaluasi pada validasi
    model.eval()
    total_val_loss = 0
    total_val_accuracy = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            total_val_loss += outputs.loss.item()

            # Menghitung akurasi
            preds = (outputs.logits > 0).float()  # Menggunakan threshold 0 untuk klasifikasi biner
            total_val_accuracy += accuracy_score(labels.cpu(), preds.cpu())

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_accuracy = total_val_accuracy / len(val_loader)
    print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss}")
    print(f"Epoch {epoch+1} - Validation Accuracy: {avg_val_accuracy}")


NameError: name 'model' is not defined